# user1 to allow user2 for accessing dataset via STS AssumeRole Feature

In [8]:
import boto3
import json
import logging
from botocore.exceptions import ClientError
logging.basicConfig(filename="boto.log", level=logging.DEBUG)

from botocore.handlers import validate_bucket_name


## (S3 resource) --> Policy --> Role --> User

## user2 will assume a role, which is created by user1 to access s3 resources owned by user1
## according to the permission policy attached to the role

endpoint_url='http://10.0.111.122:8080'

user1_iam = boto3.client('iam',
           aws_access_key_id='user1',
           aws_secret_access_key='user1pass',
           endpoint_url=endpoint_url,
           region_name='')


policy_document = "{\"Version\":\"2012-10-17\",\"Statement\":[{\"Effect\":\"Allow\",\"Principal\":{\"AWS\":[\"arn:aws:iam::odh:user/user2\"]},\"Action\":[\"sts:AssumeRole\"]}]}"



role_response = ""
#print("\n Getting Role \n")
#  Getting Role

try:
    role_response = user1_iam.get_role(
        RoleName='S3Access2'
    )
    #print(role_response)

## on first run it was blank
    
except ClientError as e:
    print("\n Creating Role \n")
#  Creating Role 
    role_response = user1_iam.create_role(
       AssumeRolePolicyDocument=policy_document,
       Path='/',
       RoleName='S3Access2',
    )
    print(role_response)


role_policy = "{\"Version\":\"2012-10-17\",\"Statement\":{\"Effect\":\"Allow\",\"Action\":\"s3:*\",\"Resource\":\"arn:aws:s3:::*\"}}"

response = user1_iam.put_role_policy(
                RoleName='S3Access2',
                PolicyName='Policy1',
                PolicyDocument=role_policy
                )

print("IAM Role and Policy has been applied, user2 can now access dataset without user1's credentials sharing")

IAM Role and Policy has been applied, user2 can now access dataset without user1's credentials sharing
